# Descriptors and Python's Dot Operator

Miki Tebeka .:. [353solutions](http://353solutions.com) .:. Python, Scientific Python and Go workshops & consulting.

## References
* [`__getattr__`](https://docs.python.org/3/reference/datamodel.html#object.__getattr__) and [`__getattribute__`](https://docs.python.org/3/reference/datamodel.html#object.__getattribute__)
* [Implementing Descriptors](https://docs.python.org/3/reference/datamodel.html#implementing-descriptors)
* [Descriptors HowTo](https://docs.python.org/3/howto/descriptor.html)

In [3]:
fp = open('/dev/random')
fp.mode

'r'

In [4]:
getattr(fp, 'mode')

'r'

In [5]:
fp.__getattribute__('mode')

'r'

In [6]:
# dot lookup first looks in object __dict__
fp.__dict__

{'mode': 'r'}

In [7]:
# vars is shorthand for __dict__
vars(fp)

{'mode': 'r'}

In [8]:
'mode' in vars(fp)

True

In [9]:
# opended in text mode, can't read
fp.read(10)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xeb in position 3: invalid continuation byte

In [10]:
fp = open('/dev/random', 'rb')
fp

<_io.BufferedReader name='/dev/random'>

In [11]:
fp.read(10)

b'm\x1b\xccH\xb8\xb5\xd8n\x93\xd3'

In [12]:
fp.mode

'rb'

In [13]:
# mode not in object
'mode' in vars(fp)

False

In [15]:
# . lookup looks next in class
'mode' in vars(fp.__class__)

True

In [16]:
fp.readlines

<function BufferedReader.readlines>

In [17]:
'readlines' in vars(fp.__class__)

False

In [18]:
fp.__class__.__bases__

(_io._BufferedIOBase,)

In [19]:
'readlines' in vars(fp.__class__.__bases__[0])

False

In [20]:
fp.__class__.__mro__

(_io.BufferedReader, _io._BufferedIOBase, _io._IOBase, object)

In [21]:
# . lookup goes over the MRO (method resolution order)
for cls in fp.__class__.__mro__:
    if 'readlines' in vars(cls):
        print(cls)
        break
else:
    print('not found')

<class '_io._IOBase'>


In [22]:
# if . lookup don't find - it raises AttributeError
fp.no_such_attr

AttributeError: '_io.BufferedReader' object has no attribute 'no_such_attr'

In [23]:
# A descriptor is class implementing __get__ (can also have __set__ and __del__)
class Desc:
    def __get__(self, inst, owner):
        print('__get__: inst: %r, owner: %r' % (inst, owner))
    def __set__(self, inst, value):
        print('__set__: inst: %r, value: %r' % (inst, value))
        
class Stock:
    symbol = Desc()
    
s = Stock()

In [24]:
s.symbol

__get__: inst: <__main__.Stock object at 0x7f7d1513a390>, owner: <class '__main__.Stock'>


In [25]:
Stock.symbol

__get__: inst: None, owner: <class '__main__.Stock'>


In [26]:
s.symbol = 'BRK.A'

__set__: inst: <__main__.Stock object at 0x7f7d1513a390>, value: 'BRK.A'


In [27]:
class Field:
    def __init__(self):
        self._value = None
        
    def __get__(self, inst, owner):
        if inst is None:
            return self
        return self._value
    
    def __set__(self, inst, value):
        self.assert_valid(value)
        self._value = value
        
    def assert_valid(self, value):
        pass
    
    
class SymbolField(Field):
    def assert_valid(self, value):
        if not str.isupper(value):
            raise ValueError('symbol must be upper case, got %r' % value)
            
class PriceField(Field):
    def assert_valid(self, value):
        if not isinstance(value, float):
            raise TypeError('price must be float, not %s' % type(value))
        if value <= 0:
            raise ValueError('price must be > 0, got %.2f' % value)
            
class Stock:
    symbol = SymbolField()
    price = PriceField()
    
    def __init__(self, symbol, price):
        self.symbol, self.price = symbol, price
    def __repr__(self):
        return 'Stock(%r, %r)' % (self.symbol, self.price)
    
brka = Stock('BRK.A', 216298.80)
brka
    

Stock('BRK.A', 216298.8)

In [28]:
brka.price = -3.2

ValueError: price must be > 0, got -3.20

In [29]:
# DON'T DO, price won't be a descriptor any more
# Stock.price = 7

In [30]:
v = Stock('V', 97.48)
v

Stock('V', 97.48)

In [31]:
# _value is class attribute, shared among instances = Fields
brka

Stock('V', 97.48)

In [32]:
# 2'nd try = store in object
class Field:
    def __get__(self, inst, owner):
        if inst is None:
            return self
        return getattr(inst, '_field')
    
    def __set__(self, inst, value):
        self.assert_valid(value)
        setattr(inst, '_field', value)
        
    def assert_valid(self, value):
        pass
    
    
class SymbolField(Field):
    def assert_valid(self, value):
        if not str.isupper(value):
            raise ValueError('symbol must be upper case, got %r' % value)
            
class PriceField(Field):
    def assert_valid(self, value):
        if not isinstance(value, float):
            raise TypeError('price must be float, not %s' % type(value))
        if value <= 0:
            raise ValueError('price must be > 0, got %.2f' % value)
            
class Stock:
    symbol = SymbolField()
    price = PriceField()
    
    def __init__(self, symbol, price):
        self.symbol, self.price = symbol, price
    
    def __repr__(self):
        return 'Stock(%r, %r)' % (self.symbol, self.price)

brka = Stock('BRK.A', 216298.80)
brka


Stock(216298.8, 216298.8)

In [33]:
# Every descriptior is using _field, need unique name
vars(brka)

{'_field': 216298.8}

In [34]:
# 3'rd try - use _attr per class
class Field:
    _attr = None
    def __get__(self, inst, owner):
        if inst is None:
            return self
        return getattr(inst, self._attr)
    
    def __set__(self, inst, value):
        self.assert_valid(value)
        setattr(inst, self._attr, value)
        
    def assert_valid(self, value):
        pass
    
    
class SymbolField(Field):
    _attr = '_symbol'
    def assert_valid(self, value):
        if not str.isupper(value):
            raise ValueError('symbol must be upper case, got %r' % value)
            
class PriceField(Field):
    _attr = '_price'
    def assert_valid(self, value):
        if not isinstance(value, float):
            raise TypeError('price must be float, not %s' % type(value))
        if value <= 0:
            raise ValueError('price must be > 0, got %.2f' % value)
            
class Stock:
    symbol = SymbolField()
    price = PriceField()
    
    def __init__(self, symbol, price):
        self.symbol, self.price = symbol, price
    
    def __repr__(self):
        return 'Stock(%r, %r)' % (self.symbol, self.price)

brka = Stock('BRK.A', 216298.80)
brka


Stock('BRK.A', 216298.8)

In [35]:
vars(brka)

{'_price': 216298.8, '_symbol': 'BRK.A'}

In [36]:
class Field:
    _attr = None
    def __get__(self, inst, owner):
        if inst is None:
            return self
        return getattr(inst, self._attr)
    
    def __set__(self, inst, value):
        self.assert_valid(value)
        setattr(inst, self._attr, value)
        
    def assert_valid(self, value):
        pass
    
    
class SymbolField(Field):
    _attr = '_symbol'
    def assert_valid(self, value):
        if not str.isupper(value):
            raise ValueError('symbol must be upper case, got %r' % value)
            
class PriceField(Field):
    _attr = '_price'
    def assert_valid(self, value):
        if not isinstance(value, float):
            raise TypeError('price must be float, not %s' % type(value))
        if value <= 0:
            raise ValueError('price must be > 0, got %.2f' % value)
            
class Stock:
    symbol = SymbolField()
    price = PriceField()
    low = PriceField()
    
    def __init__(self, symbol, price, low):
        self.symbol, self.price, self.low = symbol, price, low
    
    def __repr__(self):
        return 'Stock(%r, %r, %r)' % (self.symbol, self.price, self.low)

brka = Stock('BRK.A', 216298.80, 216297.80)
brka
# Both price and low use _price in the object

Stock('BRK.A', 216297.8, 216297.8)

In [37]:
# final version - using unique name per field
# some people use a dict inside the object to hold all the descriptor fields - so __dict__ has only one extra key
from itertools import count
class Field:
    _next_id = count().__next__
    
    def __init__(self):
        self._attr = '_%s_%d' % (self.__class__.__name__, self._next_id())
    
    def __get__(self, inst, owner):
        if inst is None:
            return self
        return getattr(inst, self._attr)
    
    def __set__(self, inst, value):
        self.assert_valid(value)
        setattr(inst, self._attr, value)
        
    def assert_valid(self, value):
        pass
    
    
class SymbolField(Field):
    _attr = '_symbol'
    def assert_valid(self, value):
        if not str.isupper(value):
            raise ValueError('symbol must be upper case, got %r' % value)
            
class PriceField(Field):
    _attr = '_price'
    def assert_valid(self, value):
        if not isinstance(value, float):
            raise TypeError('price must be float, not %s' % type(value))
        if value <= 0:
            raise ValueError('price must be > 0, got %.2f' % value)
            
class Stock:
    symbol = SymbolField()
    price = PriceField()
    low = PriceField()
    
    def __init__(self, symbol, price, low):
        self.symbol, self.price, self.low = symbol, price, low
    
    def __repr__(self):
        return 'Stock(%r, %r, %r)' % (self.symbol, self.price, self.low)

brka = Stock('BRK.A', 216298.80, 216297.80)
brka


Stock('BRK.A', 216298.8, 216297.8)

In [38]:
vars(brka)

{'_PriceField_1': 216298.8,
 '_PriceField_2': 216297.8,
 '_SymbolField_0': 'BRK.A'}

In [39]:
# Python has classmethod for alternate initializer
class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    @classmethod
    def from_str(cls, value):
        """From string in format 'x,y'"""
        x, y = map(float, value.split(","))
        return cls(x, y)
    
    def __repr__(self):
        return '%s(%r, %r)' % (self.__class__.__name__, self.x, self.y)
    
p = Point.from_str('1, 2')
p

Point(1.0, 2.0)

In [40]:
# partial demo
from functools import partial
def add(x, y):
    return x + y

add7 = partial(add, 7)
add7(110)

117

In [41]:
# Write our own classmethod using descriptor
class ClassMethod:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, inst, owner):
        return partial(self.func, owner)
    
class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    @ClassMethod
    def from_str(cls, value):
        """From string in format 'x,y'"""
        x, y = map(float, value.split(","))
        return cls(x, y)
    
    def __repr__(self):
        return '%s(%r, %r)' % (self.__class__.__name__, self.x, self.y)
    
p = Point.from_str('1, 2')
p

Point(1.0, 2.0)

In [42]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return 'Hi, I am %s. How are you?' % self.name
    
p = Person('Tim')
p.greet()

'Hi, I am Tim. How are you?'

In [43]:
# Regular methods are descriptors
p.greet.__get__

<method-wrapper '__get__' of method object at 0x7f7d15228e08>

In [46]:
meth = p.greet.__get__(p, Person)
meth()

'Hi, I am Tim. How are you?'

In [47]:
# dir show all available attributes (unless you go funky with __getattr__ or __getattribute__)
dir(fp)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_checkClosed',
 '_checkReadable',
 '_checkSeekable',
 '_checkWritable',
 '_dealloc_warn',
 '_finalizing',
 'close',
 'closed',
 'detach',
 'fileno',
 'flush',
 'isatty',
 'mode',
 'name',
 'peek',
 'raw',
 'read',
 'read1',
 'readable',
 'readinto',
 'readinto1',
 'readline',
 'readlines',
 'seek',
 'seekable',
 'tell',
 'truncate',
 'writable',
 'write',
 'writelines']

## Thank You!